In [1]:
# a. Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam



In [2]:
# b. Upload / access the dataset
data = pd.read_csv("creditcard.csv")
X = data.drop(columns = ['Class'])
y = data['Class']

scaler = StandardScaler()
X[['Time', 'Amount']] = scaler.fit_transform(X[['Time', 'Amount']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_normal = X_train[y_train == 0]



In [9]:
# c. Encoder converts it into latent representation
input_dim = X_train_normal.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(14, activation="relu")(input_layer)
encoder = Dense(7, activation="relu")(encoder)
latent = Dense(3, activation="relu")(encoder)




In [10]:
# d. Decoder networks convert it back to the original input
decoder = Dense(7, activation="relu")(latent)
decoder = Dense(14, activation="relu")(decoder)
output_layer = Dense(input_dim, activation="linear")(decoder)

autoencoder = Model(inputs=input_layer, outputs=output_layer)



In [11]:
# e. Compile the models with Optimizer, Loss, and Evaluation Metrics
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense_6 (Dense)             (None, 14)                434       
                                                                 
 dense_7 (Dense)             (None, 7)                 105       
                                                                 
 dense_8 (Dense)             (None, 3)                 24        
                                                                 
 dense_9 (Dense)             (None, 7)                 28        
                                                                 
 dense_10 (Dense)            (None, 14)                112       
                                                                 
 dense_11 (Dense)            (None, 30)                450 

In [12]:
# ===== Train the Autoencoder =====
history = autoencoder.fit(
    X_train_normal, X_train_normal,
    epochs=20,
    batch_size=256,
    shuffle=True,
    validation_split=0.2,
    verbose=1
)

# ===== Evaluate Reconstruction Error =====
X_test_pred = autoencoder.predict(X_test)
mse = np.mean(np.square(X_test_pred - X_test), axis=1)
threshold = np.mean(mse) + 3*np.std(mse)
y_pred = (mse > threshold).astype(int)

from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Epoch 1/20
622/622 [==============================] - 6s 5ms/step - loss: 0.8471 - mae: 0.5897 - val_loss: 0.6709 - val_mae: 0.5301
Epoch 2/20
622/622 [==============================] - 3s 5ms/step - loss: 0.6454 - mae: 0.5198 - val_loss: 0.6089 - val_mae: 0.5142
Epoch 3/20
622/622 [==============================] - 3s 4ms/step - loss: 0.5970 - mae: 0.5099 - val_loss: 0.5751 - val_mae: 0.5071
Epoch 4/20
622/622 [==============================] - 3s 5ms/step - loss: 0.5709 - mae: 0.5032 - val_loss: 0.5698 - val_mae: 0.5052
Epoch 5/20
622/622 [==============================] - 3s 5ms/step - loss: 0.5593 - mae: 0.4996 - val_loss: 0.5509 - val_mae: 0.4970
Epoch 6/20
622/622 [==============================] - 3s 5ms/step - loss: 0.5488 - mae: 0.4947 - val_loss: 0.5437 - val_mae: 0.4939
Epoch 7/20
622/622 [==============================] - 3s 5ms/step - loss: 0.5404 - mae: 0.4909 - val_loss: 0.5365 - val_mae: 0.4899
Epoch 8/20
622/622 [==============================] - 3s 4ms/step - loss: 0.